In [1]:

import collections as col

from emulator.env_data import high2low, fix_data
from emulator.env_factor import get_factors

from params import *


import re
import os
import numpy as np
import pandas as pd
import calendar
import matplotlib.pyplot as plt

from functools import reduce
from collections import defaultdict
from seaborn import heatmap
from copy import deepcopy
from scipy.optimize import linprog as lp
from datetime import datetime, timedelta, date
from collections import OrderedDict


#工作目录，存放代码
work_dir = os.path.dirname(os.path.abspath('__file__'))
tickdata_path = os.path.join(work_dir, 'tickdata')
factors_path = os.path.join(work_dir, 'PQFactors')
PCT_CHG_path = os.path.join(work_dir, 'PCT_CHG_NM')
concat_path = os.path.join(work_dir, 'DateConcatFactors')

def getting_factors(file_path):
    if os.path.isdir(file_path) == False:
        quotes = fix_data(os.path.join(tickdata_path, file_path))
        quotes = high2low(quotes, H2L_Freq)
        daily_quotes = high2low(quotes, D_Freq)
    
        Index = quotes.index
        High = quotes.high.values
        Low = quotes.low.values
        Close = quotes.close.values
        Open = quotes.open.values
        Volume = quotes.volume.values
        
        factors = get_factors(Index, Open, Close, High, Low, Volume, rolling = 30, drop = False)
      
        code = file_path.split('.')[0] + '.' +  file_path.split('.')[1]
        
        factors['code'] = code
        factors.to_csv(os.path.join(factors_path, file_path))
        
        pct_chg = quotes.close.pct_change()
        pct_chg.to_csv(os.path.join(PCT_CHG_path, file_path))
        
    
def concat_date_factors(file_path):
    dates = ["2020-1-1", date.today().strftime('%Y-%m-%d')]
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
    dates_dict = OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m"), None) for _ in range((end - start).days)).keys()
    dates_series = [_ for _ in dates_dict]
    
    for _ in dates_series:
        start_date = str(_) + '-01'
        last_d = calendar.monthrange(int(_.split('-')[0]), int(_.split('-')[1]))[1]
        end_date = str(_) + '-' + str(last_d)
        st = pd.to_datetime(start_date, format = '%Y-%m-%d')
        ed = pd.to_datetime(end_date,   format = '%Y-%m-%d')
        
        factors = pd.DataFrame()
        for fpath in os.listdir(factors_path)[:]:
            print(fpath)
            tmp = pd.read_csv(os.path.join(factors_path, fpath), encoding="gbk", engine='c')
            tmp = tmp.set_index(tmp.tradeTime)
            factor_date_index = pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d')
            
            if (factor_date_index > st).any == False:
                break
                
            x = tmp[(pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') > st) &\
                    (pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') < ed)].iloc[-1,:]
            factors = pd.concat([factors, x], axis = 1)
                
        factors = factors.T
        factors = factors.set_index(factors.code)
        _pathname = end_date.split('-')
        pathname = _pathname[0] + _pathname[1] + _pathname[2]
        factors.to_csv(os.path.join(concat_path, pathname))


In [2]:
#for fpath in os.listdir(tickdata_path)[:]:
#    factors = getting_factors(fpath)

In [3]:
concat_date_factors(factors_path)

2020-01-01 00:00:00 2020-01-31 00:00:00
601608.SH.csv
600008.SH.csv
600500.SH.csv
300413.SZ.csv
002340.SZ.csv
600985.SH.csv
000938.SZ.csv
002131.SZ.csv
600663.SH.csv
601106.SH.csv
002831.SZ.csv
603885.SH.csv
603369.SH.csv
300024.SZ.csv
300144.SZ.csv
600335.SH.csv
000600.SZ.csv
601985.SH.csv
600219.SH.csv
600655.SH.csv
002946.SZ.csv
600010.SH.csv
600312.SH.csv
002415.SZ.csv
000961.SZ.csv
002008.SZ.csv
600233.SH.csv
002236.SZ.csv
300015.SZ.csv
603198.SH.csv
601988.SH.csv
002311.SZ.csv
600872.SH.csv
600572.SH.csv
002004.SZ.csv
601211.SH.csv
600372.SH.csv
000932.SZ.csv
002064.SZ.csv
600869.SH.csv
000629.SZ.csv
600373.SH.csv
300315.SZ.csv
002707.SZ.csv
600908.SH.csv
600066.SH.csv
600839.SH.csv
002390.SZ.csv
600143.SH.csv
600409.SH.csv
601326.SH.csv
600025.SH.csv
000543.SZ.csv
600158.SH.csv
601933.SH.csv
002230.SZ.csv
002244.SZ.csv
002085.SZ.csv
600039.SH.csv
300257.SZ.csv
002493.SZ.csv
000002.SZ.csv
300059.SZ.csv
600277.SH.csv
600993.SH.csv
002426.SZ.csv
600031.SH.csv
600837.SH.csv
002358.S

600309.SH.csv
600026.SH.csv
300115.SZ.csv
002468.SZ.csv
002941.SZ.csv
002714.SZ.csv
601633.SH.csv
600085.SH.csv
002268.SZ.csv
600703.SH.csv
601118.SH.csv
603569.SH.csv
601377.SH.csv
600280.SH.csv
002681.SZ.csv
600845.SH.csv
603883.SH.csv
600383.SH.csv
002926.SZ.csv
002174.SZ.csv
601992.SH.csv
002128.SZ.csv
603833.SH.csv
002507.SZ.csv
002916.SZ.csv
600138.SH.csv
600827.SH.csv
300316.SZ.csv
600848.SH.csv
600038.SH.csv
600648.SH.csv
601336.SH.csv
600694.SH.csv
002672.SZ.csv
600657.SH.csv
600208.SH.csv
002056.SZ.csv
600874.SH.csv
603056.SH.csv
002081.SZ.csv
002368.SZ.csv
300383.SZ.csv
601108.SH.csv
600971.SH.csv
600171.SH.csv
300450.SZ.csv
600118.SH.csv
600745.SH.csv
600643.SH.csv
000627.SZ.csv
601001.SH.csv
000012.SZ.csv
603233.SH.csv
002212.SZ.csv
601117.SH.csv
601169.SH.csv
601229.SH.csv
600606.SH.csv
002048.SZ.csv
600588.SH.csv
600748.SH.csv
600053.SH.csv
600388.SH.csv
600639.SH.csv
600623.SH.csv
600717.SH.csv
600859.SH.csv
600690.SH.csv
601800.SH.csv
600062.SH.csv
002110.SZ.csv
002948

603993.SH.csv
600125.SH.csv
600266.SH.csv
600886.SH.csv
601878.SH.csv
300182.SZ.csv
600536.SH.csv
600487.SH.csv
603288.SH.csv
600307.SH.csv
000898.SZ.csv
600015.SH.csv
000008.SZ.csv
000703.SZ.csv
601688.SH.csv
601989.SH.csv
002352.SZ.csv
600739.SH.csv
601727.SH.csv
601198.SH.csv
002120.SZ.csv
600169.SH.csv
000725.SZ.csv
002812.SZ.csv
600260.SH.csv
600160.SH.csv
601777.SH.csv
601601.SH.csv
300287.SZ.csv
002382.SZ.csv
002465.SZ.csv
603517.SH.csv
000959.SZ.csv
002607.SZ.csv
000426.SZ.csv
600939.SH.csv
603899.SH.csv
603650.SH.csv
600729.SH.csv
601611.SH.csv
601801.SH.csv
600770.SH.csv
600598.SH.csv
002249.SZ.csv
600332.SH.csv
600064.SH.csv
000776.SZ.csv
300124.SZ.csv
300266.SZ.csv
002506.SZ.csv
002503.SZ.csv
601236.SH.csv
600267.SH.csv
000021.SZ.csv
002030.SZ.csv
600029.SH.csv
601100.SH.csv
002444.SZ.csv
601216.SH.csv
002555.SZ.csv
002422.SZ.csv
002010.SZ.csv
600393.SH.csv
002195.SZ.csv
600816.SH.csv
601689.SH.csv
601717.SH.csv
600048.SH.csv
600959.SH.csv
000630.SZ.csv
600881.SH.csv
600903

600346.SH.csv
002624.SZ.csv
601311.SH.csv
601919.SH.csv
603156.SH.csv
300376.SZ.csv
603259.SH.csv
600895.SH.csv
601003.SH.csv
600499.SH.csv
300133.SZ.csv
002414.SZ.csv
601881.SH.csv
300033.SZ.csv
600765.SH.csv
600094.SH.csv
600900.SH.csv
300134.SZ.csv
601618.SH.csv
600879.SH.csv
002044.SZ.csv
600809.SH.csv
002217.SZ.csv
600352.SH.csv
600410.SH.csv
000750.SZ.csv
002202.SZ.csv
601966.SH.csv
000728.SZ.csv
002482.SZ.csv
000401.SZ.csv
600037.SH.csv
603556.SH.csv
600036.SH.csv
600893.SH.csv
600170.SH.csv
000501.SZ.csv
002281.SZ.csv
600416.SH.csv
600660.SH.csv
000006.SZ.csv
002573.SZ.csv
002049.SZ.csv
600166.SH.csv
603866.SH.csv
600737.SH.csv
600498.SH.csv
601016.SH.csv
002544.SZ.csv
600398.SH.csv
600521.SH.csv
000983.SZ.csv
600565.SH.csv
600557.SH.csv
002690.SZ.csv
600316.SH.csv
600611.SH.csv
000988.SZ.csv
600823.SH.csv
600155.SH.csv
601928.SH.csv
002153.SZ.csv
000552.SZ.csv
001979.SZ.csv
601200.SH.csv
600022.SH.csv
601098.SH.csv
000758.SZ.csv
600362.SH.csv
600885.SH.csv
600079.SH.csv
601333

601211.SH.csv
600372.SH.csv
000932.SZ.csv
002064.SZ.csv
600869.SH.csv
000629.SZ.csv
600373.SH.csv
300315.SZ.csv
002707.SZ.csv
600908.SH.csv
600066.SH.csv
600839.SH.csv
002390.SZ.csv
600143.SH.csv
600409.SH.csv
601326.SH.csv
600025.SH.csv
000543.SZ.csv
600158.SH.csv
601933.SH.csv
002230.SZ.csv
002244.SZ.csv
002085.SZ.csv
600039.SH.csv
300257.SZ.csv
002493.SZ.csv
000002.SZ.csv
300059.SZ.csv
600277.SH.csv
600993.SH.csv
002426.SZ.csv
600031.SH.csv
600837.SH.csv
002358.SZ.csv
000062.SZ.csv
600380.SH.csv
601186.SH.csv
600004.SH.csv
603568.SH.csv
601162.SH.csv
600271.SH.csv
600760.SH.csv
000400.SZ.csv
600584.SH.csv
002508.SZ.csv
600978.SH.csv
601225.SH.csv
603888.SH.csv
002945.SZ.csv
300136.SZ.csv
002440.SZ.csv
002271.SZ.csv
603816.SH.csv
000166.SZ.csv
601838.SH.csv
300017.SZ.csv
600787.SH.csv
000860.SZ.csv
000157.SZ.csv
601899.SH.csv
600741.SH.csv
002051.SZ.csv
601766.SH.csv
600201.SH.csv
600068.SH.csv
300002.SZ.csv
002416.SZ.csv
000063.SZ.csv
000089.SZ.csv
600804.SH.csv
000999.SZ.csv
601718

600118.SH.csv
600745.SH.csv
600643.SH.csv
000627.SZ.csv
601001.SH.csv
000012.SZ.csv
603233.SH.csv
002212.SZ.csv
601117.SH.csv
601169.SH.csv
601229.SH.csv
600606.SH.csv
002048.SZ.csv
600588.SH.csv
600748.SH.csv
600053.SH.csv
600388.SH.csv
600639.SH.csv
600623.SH.csv
600717.SH.csv
600859.SH.csv
600690.SH.csv
601800.SH.csv
600062.SH.csv
002110.SZ.csv
002948.SZ.csv
600580.SH.csv
000559.SZ.csv
002280.SZ.csv
600750.SH.csv
600161.SH.csv
002818.SZ.csv
000338.SZ.csv
601111.SH.csv
000581.SZ.csv
002797.SZ.csv
601390.SH.csv
600759.SH.csv
000636.SZ.csv
002491.SZ.csv
000547.SZ.csv
002092.SZ.csv
600009.SH.csv
600458.SH.csv
300088.SZ.csv
000090.SZ.csv
600970.SH.csv
000009.SZ.csv
603077.SH.csv
601600.SH.csv
601012.SH.csv
600482.SH.csv
000768.SZ.csv
601880.SH.csv
000727.SZ.csv
600390.SH.csv
000156.SZ.csv
002155.SZ.csv
601000.SH.csv
600674.SH.csv
601006.SH.csv
000563.SZ.csv
002127.SZ.csv
002665.SZ.csv
600763.SH.csv
000970.SZ.csv
600111.SH.csv
002437.SZ.csv
600516.SH.csv
000078.SZ.csv
601019.SH.csv
002074

000426.SZ.csv
600939.SH.csv
603899.SH.csv
603650.SH.csv
600729.SH.csv
601611.SH.csv
601801.SH.csv
600770.SH.csv
600598.SH.csv
002249.SZ.csv
600332.SH.csv
600064.SH.csv
000776.SZ.csv
300124.SZ.csv
300266.SZ.csv
002506.SZ.csv
002503.SZ.csv
601236.SH.csv
600267.SH.csv
000021.SZ.csv
002030.SZ.csv
600029.SH.csv
601100.SH.csv
002444.SZ.csv
601216.SH.csv
002555.SZ.csv
002422.SZ.csv
002010.SZ.csv
600393.SH.csv
002195.SZ.csv
600816.SH.csv
601689.SH.csv
601717.SH.csv
600048.SH.csv
600959.SH.csv
000630.SZ.csv
600881.SH.csv
600903.SH.csv
600115.SH.csv
600977.SH.csv
601898.SH.csv
002142.SZ.csv
002470.SZ.csv
000686.SZ.csv
002807.SZ.csv
600633.SH.csv
600497.SH.csv
601021.SH.csv
600755.SH.csv
002176.SZ.csv
600777.SH.csv
600545.SH.csv
601005.SH.csv
601139.SH.csv
000980.SZ.csv
603806.SH.csv
300324.SZ.csv
600782.SH.csv
300159.SZ.csv
002024.SZ.csv
002434.SZ.csv
600221.SH.csv
002050.SZ.csv
000732.SZ.csv
601009.SH.csv
300027.SZ.csv
601998.SH.csv
002302.SZ.csv
002299.SZ.csv
002466.SZ.csv
300003.SZ.csv
000723